In [8]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI

In [9]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
# google_api_key = os.getenv("GOOGLE_API_KEY")

In [10]:
prompt = ChatPromptTemplate.from_template(
    "You are a helpful assistant. Answer the question based on the context provided.\n\n"
    "Context: {context}\n\n"
    "Question: {question}\n\n"
    "Answer:"
)
model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.5,
    openai_api_key=openai_api_key,
)

chain = prompt | model

print(chain.invoke({
    "context": "The capital of France is Paris. It is known for its art, fashion, and culture.",
    "question": "What is the capital of France?",
}))



content='The capital of France is Paris.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 54, 'total_tokens': 61, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bdi4xoVxIW9gLWQrP1hH3HFhRAxa4', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--3eb3084c-172b-445a-a953-40e5f99933b5-0' usage_metadata={'input_tokens': 54, 'output_tokens': 7, 'total_tokens': 61, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
